In [1]:
import pandas as pd
df = pd.read_csv("null_states.csv")
df.head()

,@timestamp,@version,_raw,_time,address,address_soundex,addresses{},city,county,cribl,...,worker.build,worker.ethnicity,worker.eye_color,worker.hair_color,worker.hair_length,worker.height,worker.implants,worker.name,worker.tattoos,zip
0,2019-12-20T18:17:17.106Z,1,"{""worker"":{""ethnicity"":""Chinese"",""breast_size""...",2020-01-23T00:40:37.000+0000,"1121 S Higley Rd, Suite 104 Mesa, AZ, 85206",S4632352,"1121 S Higley Rd, Suite 104 Mesa, AZ, 85206",NaN,Maricopa County,yes,...,Thick,Chinese,Brown,Black,Mid Back,5'0''-5'5'',0.0,Nana,0.0,85206.0
1,2019-12-20T18:17:16.940Z,1,"{""worker"":{""ethnicity"":""Asian"",""breast_size"":""...",2020-01-23T00:40:37.000+0000,"120 Elizabeth Street New York City, NY, 10013",E4213236356235,"120 Elizabeth Street New York City, NY, 10013",NaN,None,yes,...,Average,Asian,Brown,Black,Shoulder Length,5'0''-5'5'',0.0,Yuyu,0.0,10013.0
2,2019-12-20T18:17:17.105Z,1,"{""worker"":{""ethnicity"":""Asian"",""breast_size"":""...",2020-01-23T00:40:37.000+0000,"1707 SW 22nd St Miami, FL, 33145",S5323514,"1707 SW 22nd St Miami, FL, 33145",NaN,Miami-Dade County,yes,...,Average,Asian,Brown,Black,Shoulder Length,5'0''-5'5'',0.0,Amy,0.0,33145.0
3,2019-12-20T18:17:16.938Z,1,"{""worker"":{""ethnicity"":""Chinese"",""breast_size""...",2020-01-23T00:40:37.000+0000,"10S, 648 Kingery Hwy Willowbrook, IL, 60527",S52641624,"10S, 648 Kingery Hwy Willowbrook, IL, 60527",NaN,DuPage County,yes,...,Average,Chinese,Brown,Black,?,5'0''-5'5'',0.0,Cindy,0.0,60527.0
4,2019-12-20T18:17:16.122Z,1,"{""worker"":{""ethnicity"":""Chinese"",""breast_size""...",2020-01-23T00:40:37.000+0000,"506 Ferry st Little Rock, AR, 72202",F623434626,"506 Ferry st Little Rock, AR, 72202",NaN,Pulaski County,yes,...,Average,Chinese,Brown,Black,Shoulder Length,5'0''-5'5'',0.0,Helen,0.0,72202.0


In [271]:
CA_df = df[df["address"].str.contains("CA", case = True)]

In [273]:
bad_rows = []
for row in list(CA_df["address"]):
    if (row.split(","))[-2] != " CA":
        bad_rows.append(row)
        
bad_rows

['214-55 JAMAICA AVE Queens, NY, 11428',
 '8635 W. US HIGHWAY 24 CASCADE 80809 Cascade, CO, 80809',
 '214-55 JAMAICA AVE Queens, NY, 11428',
 '214-55 JAMAICA AVE Queens, NY, 11428',
 '8635 W. US HIGHWAY 24 CASCADE 80809 Cascade, CO, 80809',
 '214-55 JAMAICA AVE Queens, NY, 11428',
 '214-55 JAMAICA AVE Queens, NY, 11428',
 '214-55 JAMAICA AVE Queens, NY, 11428',
 '214-55 JAMAICA AVE Queens, NY, 11428',
 '214-55 JAMAICA AVE Queens, NY, 11428',
 '8635 W. US HIGHWAY 24 CASCADE 80809 Cascade, CO, 80809',
 '214-55 JAMAICA AVE Queens, NY, 11428',
 '214-55 JAMAICA AVE Queens, NY, 11428',
 '214-55 JAMAICA AVE Queens, NY, 11428',
 '8635 W. US HIGHWAY 24 CASCADE 80809 Cascade, CO, 80809',
 '8635 W. US HIGHWAY 24 CASCADE 80809 Cascade, CO, 80809',
 '8635 W. US HIGHWAY 24 CASCADE 80809 Cascade, CO, 80809',
 '214-55 JAMAICA AVE Queens, NY, 11428',
 '214-55 JAMAICA AVE Queens, NY, 11428',
 '8635 W. US HIGHWAY 24 CASCADE 80809 Cascade, CO, 80809',
 '214-55 JAMAICA AVE Queens, NY, 11428',
 '8635 W. US 

In [274]:
CA_df = CA_df[~CA_df["address"].isin(bad_rows)]

CA_df.to_csv("fixed_null.csv", index = False)

In [275]:
CA_og = pd.read_csv("CA_rubmaps.csv")

In [276]:
ca_full = pd.concat([CA_og, CA_df]).reset_index(drop=True)

In [277]:
ca_clean = ca_full.drop(['@timestamp', '@version', '_raw', 'address_soundex',
       'addresses{}', 'city', 'county', 'cribl','cribl_pipe', 'imb_urls{}',
              "linecount",'location_hash', 'location_id','source_id',
              'sourcetype', 'splunk_server', 'splunk_server_group',"host", "index",
       'state', 'source', "zip"], axis = 1)

In [278]:
female_names  = list(pd.read_csv("female.firstnames.csv", header = None)[0])

In [280]:
d = ca_clean[ca_clean["worker.name"].isna()]["details"].to_dict()

In [282]:
len(d)

3499

In [283]:
names = []
for name in female_names:
    if type(name) is str:
        if len(name) > 2:
            names.append(name)

In [316]:
names = [e for e in names if e not in ["SEE", "MANY", "LADY", "MISS", "LATINA",
                                       "SOON", "LATINA", "YOUNG", "LOVE", "MAN", "ELSE"]]

In [324]:
def get_key(val): 
    for key, value in d.items(): 
         if val == value: 
            return key 
  
    return "key doesn't exist"

In [334]:
names_d = {}
for index in list(d.keys()):
    details = d.get(index)
    if type(details) is str:
        names_per_details = []
        for word in details.upper().split():
            if word in names:
                if word not in names_per_details:
                        names_per_details.append(word) 
    if len(names_per_details) > 0:
        names_d[index] = names_per_details
        

In [338]:
list(names_d.values())

[['ROSE'],
 ['LISA'],
 ['HONEY'],
 ['SUNDAY'],
 ['TAMMY'],
 ['MAY'],
 ['LILLY'],
 ['LUCY'],
 ['MARK'],
 ['GOLDEN'],
 ['LINA'],
 ['JASMINE'],
 ['JANUARY'],
 ['MICHELLE'],
 ['SASHA', 'HELEN', 'LILY'],
 ['YUKI'],
 ['ISIS', 'VENUS'],
 ['JOHN', 'NGOC'],
 ['REA'],
 ['HANG'],
 ['SUMMER'],
 ['SOPHIA'],
 ['MANDI'],
 ['TINA'],
 ['MAY', 'ASIA'],
 ['SUN'],
 ['LEA', 'LISA'],
 ['DIAMOND'],
 ['GOLDEN'],
 ['MARY'],
 ['VAN'],
 ['SARAH', 'ROSE', 'JANET'],
 ['SANTA'],
 ['JACK'],
 ['OLIVE', 'LANE'],
 ['BEE'],
 ['MAY'],
 ['BELL'],
 ['MIMI'],
 ['LISA', 'TINA'],
 ['MAY'],
 ['TINA'],
 ['MAY'],
 ['CONNIE'],
 ['MAY'],
 ['MIN'],
 ['BELL'],
 ['JENNIFER'],
 ['HOPE'],
 ['HANG'],
 ['CASSIE'],
 ['CINDY'],
 ['MIN'],
 ['MAY'],
 ['MIN', 'ROSE'],
 ['LISA', 'GINA'],
 ['MAGGIE'],
 ['MAGGIE', 'LEA'],
 ['GARNET'],
 ['LISA', 'JULIE'],
 ['SIERRA'],
 ['MINDY'],
 ['SOPHIA'],
 ['NICOLE'],
 ['ANGEL'],
 ['ANGEL'],
 ['SUSAN'],
 ['AMY', 'DAWN'],
 ['VERONIKA'],
 ['MELISSA'],
 ['DEBBIE'],
 ['ANGEL'],
 ['WENDY'],
 ['TRUDY'],
 ['BEBE'],


In [340]:
ca_clean[ca_clean.index.isin(names_d.keys())]["worker.name"] = list(names_d.values())

/Users/michalgolovanevsky/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [355]:
for index in names_d.keys():
    ca_clean.at[index, "worker.name"] = names_d.get(index)

In [356]:
ca_clean[ca_clean.index.isin(names_d.keys())]["worker.name"]

180         [ROSE]
686         [LISA]
969        [HONEY]
1319      [SUNDAY]
2181       [TAMMY]
            ...   
137793       [MIA]
137977      [STAR]
138369      [TINY]
139106     [SUNNY]
139228       [AMY]
Name: worker.name, Length: 510, dtype: object

In [359]:
ca_clean.to_csv("CA_final.csv", index = False)

KeyboardInterrupt: 